## Install the dependencies

In [3]:
!pip install opencv-python opencv-contrib-python numpy


  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached numpy-2.1.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (54.8 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl (63.7 MB)
Using cached numpy-2.1.2-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)


In [4]:
import cv2
import numpy as np

# Load face detection model (Haar Cascade)
face_cascade = cv2.CascadeClassifier('project/haarcascade_frontalface_default.xml')

# Load the pre-trained models for age and gender
gender_net = cv2.dnn.readNetFromCaffe('project/gender_deploy.prototxt', 'project/gender_net.caffemodel')
age_net = cv2.dnn.readNetFromCaffe('project/age_deploy.prototxt', 'project/age_net.caffemodel')

# Define the age and gender classes
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(20-30)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']

# Function to detect faces
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    return faces

# Function to get predictions for age and gender
def predict_age_gender(face_img):
    # Prepare the face for age and gender prediction
    blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), (104.0, 177.0, 123.0), swapRB=False)
    
    # Predict gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]

    # Predict age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]
    
    return gender, age

# Initialize webcam video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect faces in the frame
    faces = detect_faces(frame)

    for (x, y, w, h) in faces:
        # Extract the face region of interest (ROI)
        face_img = frame[y:y+h, x:x+w]
        
        # Predict age and gender
        gender, age = predict_age_gender(face_img)
        
        # Display the predictions
        label = f"{gender}, {age}"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 1.0, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow('Age and Gender Detection', frame)
    
    # Break the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


2024-10-20 12:56:14.437 python[92657:7905516] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
